In [1]:
# Import packages
import pandas as pd
import geopandas as gpd
from collections import defaultdict, OrderedDict
import json

In [2]:
# Since we're working with dataframes, iloc is used for integer-location based indexing for the selected polygon
# Spatial operations: https://geopandas.org/set_operations.html

# Check if the boundaries of two polygons intersect
def shared_boundaries(gdf, id1, id2):
    g1 = gdf[gdf["dauid"] == str(id1)].geometry.iloc[0]
    g2 = gdf[gdf["dauid"] == str(id2)].geometry.iloc[0]
    return g1.length, g2.length, g1.boundary.intersection(g2.boundary).length

# Find length of boundary
def get_boundary_length(gdf, id1):
    g1 = gdf[gdf["dauid"] == str(id1)].geometry.iloc[0]
    return g1.boundary.length

In [3]:
df = pd.read_csv("data/DA Ontario Clean.csv")  # General information (id, population, area...)
df_adj = pd.read_csv("data/DA Ontario Adjacency.csv")  # Pair of adjacent territories
gdf_ontario = gpd.read_file("data/DA Ontario.gpkg")  # GeoDataFrame with the territories poligons

In [4]:
df.head()

,DAuid,DApop_2016,DAtdwell_2016,DAurdwell_2016,DAarea,CSDcode
0,35010155,457.0,209.0,189.0,65.4355,50
1,35010156,448.0,223.0,201.0,53.7040,50
2,35010157,469.0,214.0,190.0,66.4776,50
3,35010158,492.0,229.0,200.0,39.0021,50
4,35010159,517.0,227.0,213.0,35.9957,5


In [5]:
df_adj.head()

,dauid,Neighbor_dauid
0,35061341,35060145
1,35060170,35060145
2,35060148,35060145
3,35060135,35060145
4,35060144,35060145


In [6]:
gdf_ontario.head()

,dauid,csduid,DApop_2016,geometry
0,35060145,3506008,647,"MULTIPOLYGON (((7476534.349 1196783.794, 74767..."
1,35060146,3506008,535,"MULTIPOLYGON (((7477527.120 1195489.051, 74776..."
2,35060147,3506008,482,"MULTIPOLYGON (((7477129.526 1195060.197, 74770..."
3,35060148,3506008,444,"MULTIPOLYGON (((7477220.320 1195056.371, 74772..."
4,35060149,3506008,968,"MULTIPOLYGON (((7477774.920 1195083.109, 74778..."


In [7]:
template = json.loads(open("scenario.json", "r").read()) # Read json file

In [8]:
df_adj.head()

,dauid,Neighbor_dauid
0,35061341,35060145
1,35060170,35060145
2,35060148,35060145
3,35060135,35060145
4,35060144,35060145


In [9]:
df_adj[(df_adj["dauid"] == 35600266) | (df_adj["Neighbor_dauid"] == 35600266)]

,dauid,Neighbor_dauid
124433,35600267,35600266
124434,35600401,35600266
124443,35600266,35600267
125791,35600266,35600401


In [10]:
nan_rows = df[df['DApop_2016'].isnull()] # DAUIDs with missing data
zero_pop_rows = df[df["DApop_2016"] == 0] # DAUIDs with no population
# Create list of invalid DAUIDs (no population or missing data)
invalid_dauids = list(pd.concat([nan_rows, zero_pop_rows])["DAuid"])
len(invalid_dauids), len(df)

(181, 20160)

In [11]:
# Check if DAUID 35600266 is in the list of invalid DAUIDS
df_adj.iloc[124433, :]["Neighbor_dauid"] in invalid_dauids

True

In [12]:
adj_full = OrderedDict()  # Dictionary with the structure of the json output format

for ind, row in df_adj.iterrows():  # Iterate the different pair of adjacent territories
    if row["dauid"] in invalid_dauids:
        print("Invalid dauid found: ", row["dauid"])
        continue
    elif row["Neighbor_dauid"] in invalid_dauids:
        print("Invalid dauid found: ", row["Neighbor_dauid"])
        continue
    elif str(row["dauid"]) not in adj_full:
        rel_row = df[df["DAuid"] == row["dauid"]].iloc[0, :]
        pop = rel_row["DApop_2016"]
        area = rel_row["DAarea"]

        boundary_len = get_boundary_length(gdf_ontario, row["dauid"])
        state = {"population_density": pop/area, 
                 "age_divided_populations": [0.18, 0.07, 0.27, 0.31, 0.17],
                 "infected": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                 "recovered": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}
        adj_full[str(row["dauid"])] = {"cell_id": str(row["dauid"]), "state": state, "neighborhood": []}

    l1, l2, shared = shared_boundaries(gdf_ontario, row["dauid"], row["Neighbor_dauid"])
    correlation = (shared/l1 + shared/l2) / 2  # equation extracted from zhong paper (boundaries only, we don't have roads info for now)
    # correlation = math.e ** (-1/correlation)
    if correlation == 0:
        continue
    adj_full[str(row["dauid"])]["neighborhood"].append({"cell_id": str(row["Neighbor_dauid"]), "vicinity": {"correlation": correlation}})
    if ind % 1000 == 0:
        print(ind, "%.2f%%" % (100*ind/len(df_adj)))

template["cells"] = list(adj_full.values())

dauid found:  35203185
Invalid dauid found:  35200829
Invalid dauid found:  35200829
Invalid dauid found:  35200829
Invalid dauid found:  35200829
Invalid dauid found:  35200829
Invalid dauid found:  35200829
Invalid dauid found:  35200829
46000 36.50%
Invalid dauid found:  35203185
Invalid dauid found:  35203185
Invalid dauid found:  35203185
Invalid dauid found:  35203185
Invalid dauid found:  35203185
47000 37.30%
Invalid dauid found:  35203717
Invalid dauid found:  35203717
Invalid dauid found:  35203717
Invalid dauid found:  35203717
Invalid dauid found:  35203717
Invalid dauid found:  35203717
Invalid dauid found:  35204899
Invalid dauid found:  35203185
Invalid dauid found:  35204854
Invalid dauid found:  35204878
Invalid dauid found:  35240903
48000 38.09%
Invalid dauid found:  35204258
Invalid dauid found:  35204878
Invalid dauid found:  35201461
49000 38.88%
Invalid dauid found:  35204854
Invalid dauid found:  35204854
Invalid dauid found:  35204854
Invalid dauid found:  3520

In [13]:
adj_full_json = json.dumps(template, indent=4, sort_keys=False)  # Dictionary to string (with indentation=4 for better formatting)

In [14]:
with open("ontario_cadmium_w6.json", "w") as f:
    f.write(adj_full_json)